In [ ]:
import numpy as np
#Scikit learn
from sklearn.feature_selection import VarianceThreshold
#from sklearn.svm import LinearSV
from sklearn import linear_model 
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import PolynomialFeatures
import ipynb
from ipynb.fs.full.joinFeatures import mergedData
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

#Bokeh, interactive figures
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html
#print(os.listdir("./"))
output_notebook()
plt.rc("figure", figsize=(25,12))
plt.rc("font", size=10)
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'




def descriptions1(modelName, results, predictions):   
    print(modelName)
    print(results.summary())
    print(predictions)

def mse1(obs1, col1):
    return(np.sum(np.square(obs1 - col1)))

def mae1(obs1, col1):
    return(np.sum(np.abs(obs1 - col1)))    

    
def myLasso(trainData, testData, allCols, myAlpha, myMinValue):
    train1 = trainData[allCols]
    train2 = train1.dropna()
    features = list(filter(lambda x : x != 'Yield', allCols))
    
    X = train2[features].values
    y = train2["Yield"].values
    
    poly = PolynomialFeatures(3)
    polyX = poly.fit_transform(X)
    
    features = pd.DataFrame(polyX, columns=poly.get_feature_names(features))
    features.columns = [w.replace(' ', '_') for w in features.columns]
    features.columns = [w.replace('^2', 'sq') for w in features.columns]
    features.columns = [w.replace('^3', 'cube') for w in features.columns]
    
    features["Year"] = features["Year"].astype(int)
    
    clf = linear_model.Lasso(alpha=myAlpha, normalize=False, max_iter=100000, tol=1e-100).fit(polyX, y)
    
    dict1 = dict(zip(features, clf.coef_))
    dict2 = dict((k, v) for k, v in dict1.items() if np.abs(v) >= myMinValue)    
    
    #print("best features:", dict2)
    
    features["Yield"] = y
    
    nms = list(dict2.keys())
    nms3 = [w.replace(' ', '_') for w in nms]
    nms3 = [w.replace('^2', 'sq') for w in nms3]
    nms3 = [w.replace('^3', 'cube') for w in nms3]
  
    nms = '+'.join(nms3)
    
    nms2 = 'Yield ~ ' + nms
    
    model = smf.ols(nms2 , data=features).fit()
    
    
    ####### Transform test data
    test1 = testData[allCols]
    test2 = test1.dropna()
    testfeatures = list(filter(lambda x : x != 'Yield', test2))
    
    X = test2[testfeatures].values
    
    
    polyX = poly.fit_transform(X)
    
    featurestest = pd.DataFrame(polyX, columns=poly.get_feature_names(testfeatures))
  
    featurestest.columns = [w.replace(' ', '_') for w in featurestest.columns]
    featurestest.columns = [w.replace('^2', 'sq') for w in featurestest.columns]
    featurestest.columns = [w.replace('^3', 'cube') for w in featurestest.columns]
    featurestest["Year"] = featurestest["Year"].astype(int)
    
    test3 = featurestest[nms3]  #has to be originally identified features
    
    predictions =  model.predict(test3) # predict out of sample
    
    return(model, predictions)


def bokehFig(data):
    p = figure(title="Year Yields", y_axis_type="linear", plot_height = 500,
               tools = TOOLS, plot_width = 800)
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Yields'
    p.circle(2016, data.Yield.max(), size = 10, color = 'red')

    p.line(data.Year, data.Yield,line_color="blue", line_width = 2)
    p.line(data.Year, data.Yield, legend_label="Yield", line_color="blue", line_width = 1)
    p.legend.location = "top_left"
    p.select_one(HoverTool).tooltips = [('year', '@x'),('yield', '@y'),]
    output_file("line_chart.html", title="Line Chart")
    return(p)

    